In [1]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib.pyplot as plt
import os
import numpy as np
from gc import callbacks
import scipy.linalg as la
from sklearn import preprocessing
from distfit import distfit
from sklearn.decomposition import PCA
import sys 
from scipy.optimize import curve_fit
import scipy.constants as constants 


In [2]:
# read the file
# mortality
zipped = True
if zipped:
    mdf_path = r'data/mdf_final.csv'
    mdf = pd.read_csv(mdf_path, low_memory=False)
    mdf = mdf.drop(columns='Unnamed: 0')
    
    pdf_path = r'data\pdf_final.csv'
    pdf = pd.read_csv(pdf_path, low_memory=False)
    pdf = pdf.drop(columns=['Lb', 'Unnamed: 0'])
    
    lnmrdf = pd.read_csv(r'data/lnmr_final.csv', low_memory=False)
    lnmrdf = lnmrdf.drop(columns='Unnamed: 0')
    
    lnfrdf = pd.read_csv(r'data/lnfr_final.csv', low_memory=False)
    lnfrdf = lnfrdf.drop(columns='Unnamed: 0')
    
    
else:
    mortality_path_7 = r'data/MortIcd7.csv'
    mortality_data_7 = pd.read_csv(mortality_path_7, low_memory=False)

    mortality_path_8 = r'data/MortIcd8.csv'
    mortality_data_8 = pd.read_csv(mortality_path_8, low_memory=False)

    mortality_path_9 = r'data/Morticd9.csv'
    mortality_data_9 = pd.read_csv(mortality_path_9, low_memory=False)

    mortality_path_2002 = r'data/Morticd10_part1.csv'
    mortality_data_2002 = pd.read_csv(mortality_path_2002, low_memory=False)

    mortality_path_2007 = r'data/Morticd10_part2.csv'
    mortality_data_2007 = pd.read_csv(mortality_path_2007, low_memory=False)

    mortality_path_2012 = r'data/Morticd10_part3.csv'
    mortality_data_2012 = pd.read_csv(mortality_path_2012, low_memory=False)

    mortality_path_2016 = r'data/Morticd10_part4.csv'
    mortality_data_2016 = pd.read_csv(mortality_path_2016, low_memory=False)

    mdf = pd.concat([mortality_data_7, mortality_data_8, mortality_data_9, mortality_data_2002, mortality_data_2007, mortality_data_2012, mortality_data_2016], ignore_index=True)
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)

    # population
    pop_path = r'data/pop.csv'
    pdf = pd.read_csv(pop_path, low_memory=False)
    dropped = False
    if not dropped:
        undesired_population_columns = ['Admin1', 'SubDiv', 'Sex', 'Frmat', 'Lb'] 
        pdf = pdf.drop(columns = undesired_population_columns)

# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']

In [3]:
# needs to swap out the nah columns into averages 
done = True
if not done:
    not_used_country_list = []
    country_num_count = 0
    not_useful = False
    for key in country_code_map:
        country_num_count += 1
        display(country_num_count / len(country_code_map))
        country_mdf = mdf[mdf['Country'] == key]
        
        if country_mdf.empty:
            not_used_country_list.append(key)
        
        else:
            min_year = int(country_mdf['Year'].min())
            max_year = int(country_mdf['Year'].max()) 
            for i in range (min_year, max_year + 1):
                
                if not_useful:
                    break
                
                mask = (mdf['Country'] == key) & (mdf['Year'] == i)
                curr_Year = i

                for c in mdf.columns.values:
                    
                    if not_useful:
                        break
                    
                    average = mdf.loc[mask, c].mean()   
                    
                    max_iter = max_year - curr_Year
                    count = 0 
                    while np.isnan(average):
                        curr_Year += 1
                        count += 1
                        mask_alt = (mdf['Country'] == key) & (mdf['Year'] == curr_Year) 
                        average = mdf.loc[mask_alt, c].mean()   
                        
                        if count > max_iter:
                            not_useful = True
                            break 
                        
                        
                    mdf.loc[mask, c] = mdf.loc[mask, c].fillna(average)
                    curr_Year = i 
                    
            if not_useful:
                display(key)
                not_used_country_list.append(key)
                index = mdf[mdf['Country'] == key].index
                mdf = mdf.drop(index)
                not_useful = False
                
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)            
    mdf = mdf.groupby(['Country', 'Year'], as_index=True).agg("sum").reset_index()
    
    # building the potential country list 
    country_list = list(set(mdf['Country']))
    country_list.sort()
    len(country_list) 
        
            

        

In [4]:
# build pdf
done = True
if not done:
    not_used_pop_list = []
    country_num_count = 0
    not_useful = False
    for key in country_list:
        
            country_num_count += 1
            display(country_num_count / len(country_list))
            
            country_pdf = pdf[pdf['Country'] == key]
            
            if country_pdf.empty:
                not_used_pop_list.append(key)
            
            else:
                min_year = int(country_pdf['Year'].min())
                max_year = int(country_pdf['Year'].max()) 
                
                if min_year == max_year:
                    if (country_pdf.isnull().sum().sum() > 0):
                        not_useful = True
                    else:
                        continue
                
                for i in range (min_year, max_year + 1):
                    
                    if not_useful:
                        break
                    
                    mask = (pdf['Country'] == key) & (pdf['Year'] == i)
                    curr_Year = i

                    for c in pdf.columns.values:
                        
                        if not_useful:
                            break
                        
                        average = pdf.loc[mask, c].mean()   
                        
                        max_iter = max_year - curr_Year
                        count = 0 
                        while np.isnan(average):
                            curr_Year += 1
                            count += 1
                            mask_alt = (pdf['Country'] == key) & (pdf['Year'] == curr_Year)
                            average = pdf.loc[mask_alt, c].mean()   
                            
                            if count > max_iter:
                                not_useful = True
                                break 
                            
                            
                        pdf.loc[mask, c] = pdf.loc[mask, c].fillna(average)
                        curr_Year = i 
                        
                if not_useful:
                    display(key)
                    not_used_pop_list.append(key)
                    index = pdf[pdf['Country'] == key].index
                    pdf = pdf.drop(index)
                    not_useful = False
                    
    # delete the unused countries 
    for key in not_used_pop_list:
        country_list.remove(key)
        
    # finalizing pdf 
    pdf = pdf[pdf['Country'].isin(country_list)]
    pdf = pdf.groupby(['Country', 'Year'], as_index=True).agg("sum").reset_index()
    pdf.to_csv(r'data/pdf_final.csv')
    
    # finalizing mdf 
    mdf = mdf[mdf['Country'].isin(country_list)]
    mdf.to_csv(r'data/mdf_final.csv')
                    
                
        
    

In [5]:
# create the log mortality rate graph 

done = True
if not done:
    
    lnmr_dict = {}
    lnmr_dict['Country'] = []
    lnmr_dict['Year'] = []
    for i in range (6, 27):
        lnmr_column = f'Log Mortality{i}'
        lnmr_dict[lnmr_column] = []
    
    lnmrdf = pd.DataFrame(lnmr_dict)
    
    country_list = list(set(mdf['Country']))
    country_list.sort()
    
    country_num_count = 0
    for key in country_list:
        
        country_num_count += 1
        display(country_num_count / len(country_list))
        
        country_mdf = mdf[mdf['Country'] == key]
        min_year = int(country_mdf['Year'].min())
        max_year = int(country_mdf['Year'].max()) 
        
        for i in range (min_year, max_year + 1):
            
            m_mask = (mdf['Country'] == key) & (mdf['Year'] == i)
            p_mask = (pdf['Country'] == key) & (pdf['Year'] == i)
            
            if pdf[p_mask].empty or mdf[m_mask].empty:
                continue
            else:
                lnmrdf.loc[lnmrdf.shape[0]] = ['None'] * lnmrdf.shape[1]
                lnmrdf.loc[lnmrdf.shape[0]-1, ['Country']] = key
                lnmrdf.loc[lnmrdf.shape[0]-1, ['Year']] = i
            
            for c in range(6, 27):
                
                if c == 6:
                    deaths = 0
                    population = 0
                    for j in range (2, 7):
                        m_column = f'Deaths{c}'
                        p_column = f'Pop{c}'
                        lnmr_column = f'Log Mortality{c}'
                        try:
                            deaths += mdf[m_mask][m_column].values.astype('float')[0]
                            population += pdf[p_mask][p_column].values.astype('float')[0]
                        except:
                            display(key)
                            display(i)
                            display(c)
                            callbacks()
                else:
                    m_column = f'Deaths{c}'
                    p_column = f'Pop{c}'
                    lnmr_column = f'Log Mortality{c}'
                    try:
                        deaths = mdf[m_mask][m_column].values.astype('float')[0]
                        population = pdf[p_mask][p_column].values.astype('float')[0]
                    except:
                        display(key)
                        display(i)
                        display(c)
                        callbacks()
                try:
                    ratio = deaths / population
                    lnmrdf.loc[lnmrdf.shape[0]-1, lnmr_column] = np.log(ratio)
                except:
                    display(key)
                    display(i)
                    display(c)
                    callbacks()
    
    lnmrdf.to_csv(r'data/lnmr_final.csv')

In [12]:
# fit the mortality model 
from numpy import inf


done = False
pred_year = 2050
each_country_num = 50
if not done:
    
    country_list = list(set(lnmrdf['Country']))
    country_list.sort()
    mort_d = {}
    
    for key in country_list:
        
        try: 
        
            display(key)
            c_s = []
            
            country_lnmrdf = lnmrdf[lnmrdf['Country'] == key]
            min_year = int(country_lnmrdf['Year'].min())
            max_year = int(country_lnmrdf['Year'].max())
            
            ma = country_lnmrdf[country_lnmrdf['Year'] > min_year].drop(columns=['Country', 'Year']).values
            mb = country_lnmrdf[country_lnmrdf['Year'] < max_year].drop(columns=['Country', 'Year']).values

            m = np.nan_to_num(ma - mb)
            scaler = preprocessing.StandardScaler().fit(m)
            m_scaled = np.nan_to_num(scaler.transform(m))
            u, singular, V_transpose = np.linalg.svd(m_scaled)
            singular[singular == inf] = sys.float_info.max
            dist = distfit()
            # gamma distribution 
            dist.fit_transform(singular)
            # loading vectors
            pca = PCA(n_components=1)
            transformed_data = pca.fit(m_scaled).transform(m_scaled)
            eigenValues = pca.explained_variance_ratio_
            # loading = pca.components_.T * np.sqrt(pca.explained_variance_)
            loadings = pd.DataFrame(pca.components_.T)
            lv = np.matrix(loadings[0].values)
            y = int((pred_year - max_year)) 
            for i in range (each_country_num):    
                m_new = np.matrix(dist.generate(y)).reshape(y, 1)
                pred = m_new * lv
                c_s.append(pred)
                
            mort_d[key] = c_s
        
        except:
            continue

2140

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 0.00613154] [loc=4.948 scale=3.558]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 0.00384375] [loc=0.000 scale=4.948]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 0.0291817] [loc=-0.000 scale=0.000]
[distfit] >INFO> [dweibull  ] [0.02 sec] [RSS: 0.00543852] [loc=4.421 scale=2.790]
[distfit] >INFO> [t         ] [0.15 sec] [RSS: 0.00557613] [loc=4.785 scale=3.569]
[distfit] >INFO> [genextreme] [0.06 sec] [RSS: 0.00177029] [loc=3.296 scale=2.733]
[distfit] >INFO> [gamma     ] [0.01 sec] [RSS: 0.00100171] [loc=-0.932 scale=2.363]
[distfit] >INFO> [lognorm   ] [0.04 sec] [RSS: 0.00143144] [loc=-3.711 scale=7.980]
[distfit] >INFO> [beta      ] [0.13 sec] [RSS: 0.00251999] [loc=-0.000 scale=25.389]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 0.0118355] [loc=0.000 scale=14.740]
[distfit] >INFO> [loggamma  ] [0.05 sec] [RSS: 0.00618902] [loc=-1145.470 scale=153.872]
[distfit] >INFO> Compute conf

3080

[distfit] >INFO> fit
[distfit] >INFO> transform


3160

...

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 0.00645844] [loc=5.538 scale=5.589]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 0.000877454] [loc=0.000 scale=5.538]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 0.000890916] [loc=-817.818 scale=817.818]
[distfit] >INFO> [dweibull  ] [0.04 sec] [RSS: 0.00641266] [loc=4.426 scale=3.307]
[distfit] >INFO> [t         ] [0.13 sec] [RSS: 0.00424331] [loc=4.126 scale=2.681]
[distfit] >INFO> [genextreme] [0.06 sec] [RSS: 0.000842141] [loc=2.961 scale=2.589]
[distfit] >INFO> [gamma     ] [0.12 sec] [RSS: 0.0126954] [loc=-0.000 scale=1.733]
[distfit] >INFO> [lognorm   ] [0.07 sec] [RSS: 0.000810442] [loc=-0.953 scale=4.947]
[distfit] >INFO> [beta      ] [0.21 sec] [RSS: 0.00102104] [loc=-0.000 scale=419.773]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 0.0161162] [loc=0.000 scale=26.819]
[distfit] >INFO> [loggamma  ] [0.09 sec] [RSS: 0.00702519] [loc=-1503.287 scale=211.161]
[distfit] >INFO> Com

4300

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 0.00251301] [loc=6.588 scale=4.190]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 0.00966899] [loc=0.000 scale=6.588]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 0.0273103] [loc=-0.000 scale=0.000]
[distfit] >INFO> [dweibull  ] [0.03 sec] [RSS: 0.00108227] [loc=6.028 scale=3.181]
[distfit] >INFO> [t         ] [0.20 sec] [RSS: 0.00103173] [loc=6.035 scale=2.938]
[distfit] >INFO> [genextreme] [0.13 sec] [RSS: 0.000490402] [loc=4.739 scale=3.277]
[distfit] >INFO> [gamma     ] [0.02 sec] [RSS: 0.000749226] [loc=-2.379 scale=1.887]
[distfit] >INFO> [lognorm   ] [0.08 sec] [RSS: 0.000552686] [loc=-5.820 scale=11.770]
[distfit] >INFO> [beta      ] [0.09 sec] [RSS: 0.000760475] [loc=-2.323 scale=8834058.441]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 0.0120886] [loc=0.000 scale=19.648]
[distfit] >INFO> [loggamma  ] [0.10 sec] [RSS: 0.00280712] [loc=-1033.747 scale=147.105]
[distfit] >INFO> Co

5020

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 0.0113574] [loc=5.066 scale=6.020]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 0.00121004] [loc=0.000 scale=5.066]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 0.00153196] [loc=-36.873 scale=36.873]
[distfit] >INFO> [dweibull  ] [0.07 sec] [RSS: 0.00883425] [loc=3.944 scale=3.273]
[distfit] >INFO> [t         ] [0.08 sec] [RSS: 0.00508366] [loc=3.566 scale=2.431]
[distfit] >INFO> [genextreme] [0.09 sec] [RSS: 0.00172656] [loc=2.406 scale=2.260]
[distfit] >INFO> [gamma     ] [0.08 sec] [RSS: 0.01152] [loc=-0.000 scale=1.733]
[distfit] >INFO> [lognorm   ] [0.06 sec] [RSS: 0.00166498] [loc=-0.583 scale=3.905]
[distfit] >INFO> [beta      ] [0.13 sec] [RSS: 0.00173649] [loc=-0.000 scale=1673.871]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 0.0240908] [loc=0.000 scale=29.251]
[distfit] >INFO> [loggamma  ] [0.10 sec] [RSS: 0.011899] [loc=-2594.636 scale=327.890]
[distfit] >INFO> Compute confi

In [6]:
# construct log fertility graph 
done = True
if not done:
    
    lnfr_dict = {}
    lnfr_dict['Country'] = []
    lnfr_dict['Year'] = []
    lnfr_dict['Log Fertility'] = []
    
    lnfrdf = pd.DataFrame(lnfr_dict)

    for key in mort_d:
        
        country_pdf = pdf[pdf['Country'] == key]
        country_mdf = mdf[(mdf['Country'] == key)]
        country_mdf = country_mdf[(country_mdf['Year'].isin(country_pdf['Year']))]
        min_year = np.max([int(country_mdf['Year'].min()), int(country_pdf['Year'].min())])
        max_year = np.min([int(country_mdf['Year'].max()), int(country_pdf['Year'].max())])

        for year in range(min_year, max_year):
            
            try:
            
                lnfrdf.loc[lnfrdf.shape[0]] = ['None'] * lnfrdf.shape[1]
                lnfrdf.loc[lnfrdf.shape[0]-1, ['Country']] = key
                lnfrdf.loc[lnfrdf.shape[0]-1, ['Year']] = year
                a = country_pdf[country_pdf['Year'] == year+1]['Pop1'].values[0]
                b = country_pdf[country_pdf['Year'] == year]['Pop1'].values[0]
                d = country_mdf[country_mdf['Year'] == year + 1]['Deaths1'].values[0]
                yp = (a - b + d) / b 
                lnfrdf.loc[lnfrdf.shape[0]-1, 'Log Fertility'] = np.log(yp)
            
            except:
                continue
    
    lnfrdf.to_csv(r'data/lnfr_final.csv')
    

In [13]:
# fit the fertility model 
done = False
if not done:
    country_list = list(set(lnfrdf['Country']))
    country_list.sort()
    fert_d = {}
    
    for key in country_list:
        display(key)
        c_s = []
        country_lnmrdf = lnmrdf[lnmrdf['Country'] == key]
        max_year = int(country_lnmrdf['Year'].max())
        p = np.nan_to_num(lnfrdf[lnfrdf['Country'] == key]['Log Fertility'].values)
        cp = p[1:] - p[:-1]
        dist = distfit()
        # gamma distribution 
        dist.fit_transform(cp)
        
        
        y = int((pred_year - max_year))
        
        for i in range (each_country_num):    
            pred = np.matrix(dist.generate(y)).reshape(y, 1)
            c_s.append(pred)
        
    
        fert_d[key] = c_s
        
        
    

2140

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 3.07139] [loc=0.006 scale=0.311]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 8.59493] [loc=-0.729 scale=0.734]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 8.59493] [loc=-67108864.729 scale=67108864.000]
[distfit] >INFO> [dweibull  ] [0.06 sec] [RSS: 0.722594] [loc=0.004 scale=0.159]
[distfit] >INFO> [t         ] [0.21 sec] [RSS: 0.597556] [loc=-0.020 scale=0.122]
[distfit] >INFO> [genextreme] [0.12 sec] [RSS: 3.28401] [loc=-0.111 scale=0.308]
[distfit] >INFO> [gamma     ] [0.08 sec] [RSS: 3.04802] [loc=-4.593 scale=0.021]
[distfit] >INFO> [lognorm   ] [0.10 sec] [RSS: 3.04401] [loc=-6.740 scale=6.739]
[distfit] >INFO> [beta      ] [0.10 sec] [RSS: 3.18004] [loc=-2.188 scale=6.005]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 6.52209] [loc=-0.729 scale=1.533]
[distfit] >INFO> [loggamma  ] [0.06 sec] [RSS: 3.12517] [loc=-71.516 scale=10.266]
[distfit] >INFO> Compute confidence intervals

3160

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 63.0996] [loc=-0.011 scale=0.156]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 111.815] [loc=-0.440 scale=0.430]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 111.815] [loc=-33554432.440 scale=33554432.000]
[distfit] >INFO> [dweibull  ] [0.04 sec] [RSS: 19.7453] [loc=-0.011 scale=0.091]
[distfit] >INFO> [t         ] [0.08 sec] [RSS: 10.7476] [loc=-0.011 scale=0.042]
[distfit] >INFO> [genextreme] [0.12 sec] [RSS: 65.9373] [loc=-0.070 scale=0.154]
[distfit] >INFO> [gamma     ] [0.07 sec] [RSS: 62.9342] [loc=-1.865 scale=0.013]
[distfit] >INFO> [lognorm   ] [0.05 sec] [RSS: 62.8722] [loc=-2.475 scale=2.460]
[distfit] >INFO> [beta      ] [0.09 sec] [RSS: 63.1055] [loc=-1.591 scale=2968.083]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 98.5699] [loc=-0.440 scale=0.911]
[distfit] >INFO> [loggamma  ] [0.06 sec] [RSS: 63.6231] [loc=-35.030 scale=5.053]
[distfit] >INFO> Compute confidence interva

3400

...

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 82.018] [loc=0.007 scale=0.142]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 158.076] [loc=-0.471 scale=0.478]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 158.076] [loc=-67108864.471 scale=67108864.000]
[distfit] >INFO> [dweibull  ] [0.06 sec] [RSS: 31.8346] [loc=0.009 scale=0.050]
[distfit] >INFO> [t         ] [0.08 sec] [RSS: 8.60138] [loc=0.004 scale=0.033]
[distfit] >INFO> [genextreme] [0.08 sec] [RSS: 93.6019] [loc=-0.048 scale=0.150]
[distfit] >INFO> [gamma     ] [0.08 sec] [RSS: 79.7323] [loc=-1.439 scale=0.013]
[distfit] >INFO> [lognorm   ] [0.04 sec] [RSS: 79.1997] [loc=-1.804 scale=1.806]
[distfit] >INFO> [beta      ] [0.08 sec] [RSS: 80.4346] [loc=-1.296 scale=202574.324]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 151.317] [loc=-0.471 scale=1.281]
[distfit] >INFO> [loggamma  ] [0.05 sec] [RSS: 83.0453] [loc=-45.601 scale=6.085]
[distfit] >INFO> Compute confidence intervals

4300

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 51.3876] [loc=-0.004 scale=0.645]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 65.2636] [loc=-3.624 scale=3.621]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 65.2636] [loc=-536870915.624 scale=536870912.000]
[distfit] >INFO> [dweibull  ] [0.03 sec] [RSS: 10.3919] [loc=-0.002 scale=0.093]
[distfit] >INFO> [t         ] [0.05 sec] [RSS: 4.03772] [loc=0.006 scale=0.051]
[distfit] >INFO> [genextreme] [0.10 sec] [RSS: 56.0246] [loc=-0.258 scale=0.872]
[distfit] >INFO> [gamma     ] [0.06 sec] [RSS: 51.5053] [loc=-27.285 scale=0.015]
[distfit] >INFO> [lognorm   ] [0.11 sec] [RSS: 51.3893] [loc=-236.231 scale=236.226]
[distfit] >INFO> [beta      ] [0.09 sec] [RSS: 51.4041] [loc=-159.804 scale=260.829]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 63.8733] [loc=-3.624 scale=7.207]
[distfit] >INFO> [loggamma  ] [0.05 sec] [RSS: 51.435] [loc=-152.240 scale=21.675]
[distfit] >INFO> Compute confidence

5020

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 37.9964] [loc=-0.008 scale=0.663]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 49.4577] [loc=-3.628 scale=3.619]
[distfit] >INFO> [pareto    ] [0.00 sec] [RSS: 49.4577] [loc=-1073741827.628 scale=1073741824.000]
[distfit] >INFO> [dweibull  ] [0.03 sec] [RSS: 11.8524] [loc=-0.013 scale=0.129]
[distfit] >INFO> [t         ] [0.05 sec] [RSS: 3.34042] [loc=-0.001 scale=0.060]
[distfit] >INFO> [genextreme] [0.06 sec] [RSS: 41.5256] [loc=-0.269 scale=0.878]
[distfit] >INFO> [gamma     ] [0.05 sec] [RSS: 38.0525] [loc=-33.815 scale=0.013]
[distfit] >INFO> [lognorm   ] [0.14 sec] [RSS: 37.9918] [loc=-232.848 scale=232.838]
[distfit] >INFO> [beta      ] [0.12 sec] [RSS: 38.009] [loc=-120.068 scale=291.498]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 48.2498] [loc=-3.628 scale=7.262]
[distfit] >INFO> [loggamma  ] [0.08 sec] [RSS: 38.0322] [loc=-178.930 scale=24.737]
[distfit] >INFO> Compute confide

In [10]:
x = pdf['Year'].values
x

array([1955, 1956, 1957, ..., 2019, 2020, 2021], dtype=int64)

In [14]:
def function (x, k, b):
    y = b * np.exp(k * x)
    return y

In [15]:
# construct the final graph
done = False
first = True
if not done:
    
    for key in mort_d:
        try:
            country_mdf = mdf[mdf['Country'] == key]
            country_pdf = pdf[pdf['Country'] == key]
            country_lnmrdf = lnmrdf[lnmrdf['Country'] == key]
            country_lnfrdf = lnfrdf[lnfrdf['Country'] == key]
                    
            max_year = np.min([(country_lnmrdf['Year'].max(), country_lnfrdf['Year'].max(), country_pdf['Year'].max())])
            country_pdf.plot(kind='scatter',x='Year',y='Pop1',color='red')    
                    
            year_range = int((pred_year - max_year))
            curr_Year_array = []
            total_pop_array = []
            for i in mort_d[key]:
                for j in fert_d[key]:
                    
                    max_lnmrdf_row = country_lnmrdf[country_lnmrdf['Year'] == max_year].drop(columns=['Country', 'Year']).values
                    max_lnfrdf_row = country_lnfrdf[country_lnfrdf['Year'] == max_year].drop(columns=['Country', 'Year']).values
                    max_pdf_row = country_pdf[country_pdf['Year'] == max_year].drop(columns=['Country', 'Year', 'Pop1']).values[0]
                    curr_Year = max_year
                    population_array = []
                    population_array.append(max_pdf_row[:5].sum())
                    
                    for ele in max_pdf_row[5:]:
                        population_array.append(ele)
                    
                    population_array = np.array(population_array)
                    
                    for year in range (0, year_range-1):
                        
                        next_lnmrdf = i[year]
                        next_lnfrdf = j[year]
                        
                        max_lnmrdf_row += next_lnmrdf
                        actually_mr = np.diag(np.exp(max_lnmrdf_row)[0]) 
                        max_lnfrdf_row += next_lnfrdf
                        actually_fr = np.exp(max_lnfrdf_row)[0][0]

                        total_pop = population_array.sum()
                
                        deaths = population_array * actually_mr
                        deaths = np.nan_to_num(deaths.diagonal()) 
                        total_deaths = deaths.sum() 
                        
                        births = total_pop * actually_fr
                        
                        population_array[0] += births
                        population_array -= deaths
                        
                        for ele in range(len(population_array)):
                            if population_array[ele] < 0:
                                population_array[ele] = 0
                        
                        aged = population_array / 5
                        population_array -= aged
                        
                        for ele in range (len(aged)-1):
                            population_array[ele+1] += aged[ele]
                            
                        population_array = np.nan_to_num(population_array)  
                        population_array[population_array == inf] = 0
                        
                        
                        total_pop = population_array.sum()
                        curr_Year += 1
                        
                        if total_pop == inf or total_pop == 0:
                            break
                        else:
                            curr_Year_array.append(curr_Year)
                            total_pop_array.append(total_pop)
                            
                        # plt.scatter(curr_Year, total_pop, color='blue')
                
        
            
    


            total_pop_array = np.array(total_pop_array)
            curr_Year_array = np.array(curr_Year_array)

            q1 = np.quantile(total_pop_array, 0.25)
            q3 = np.quantile(total_pop_array, 0.75)

            iqr = q3 - q1
            upperBound = q3 + 1.5 * iqr
            lowerBound = q1 - 1.5 * iqr

            indices = np.where((total_pop_array < lowerBound) | (total_pop_array > upperBound))[0]
            indices_filter = [z for z,n in enumerate(total_pop_array) if z not in indices]

            fixed_total_pop_array = total_pop_array[indices_filter]
            fixed_curr_Year_array = curr_Year_array[indices_filter]

            plt.scatter(fixed_curr_Year_array, fixed_total_pop_array, color='blue', alpha=0.5, label = 'data')
            
            min_year = country_pdf['Year'].min()
            xdata = np.concatenate(country_pdf['Year'].values, curr_Year_array) - min_year
            display(xdata)
            ydata = np.concatenate(country_pdf['Pop1'].values, total_pop_array) 
            
            parameters, covariance = curve_fit(function, xdata, ydata)
            fit_K = parameters[0]
            fit_B = parameters[1]
            fit_y = function(curr_Year_array - max_year, fit_K, fit_B)
            plt.plot(curr_Year_array, fit_y, '-', label = 'fit')
            
            
            name = country_code_map[key]
            path = f'Images/{name}_{key}.png'
            plt.savefig(path)
            
            break
            
            
            
            
        except:
            continue
    
                    
                    
                    
                    
                    
                                        
                    
        

In [ ]:
using_dict = [2140,3400,4010,4020,4030,4045,4140,4180,4190,4220,4272,4273,4280,4290,4300,5020]

In [ ]:
key = 2140
country_mdf = mdf[mdf['Country'] == key]
country_pdf = pdf[pdf['Country'] == key]
country_lnmrdf = lnmrdf[lnmrdf['Country'] == key]
country_lnfrdf = lnfrdf[lnfrdf['Country'] == key]
        
max_year = np.min([(country_lnmrdf['Year'].max(), country_lnfrdf['Year'].max(), country_pdf['Year'].max())])
country_pdf.plot(kind='scatter',x='Year',y='Pop1',color='red')    
        
year_range = int((pred_year - max_year))
curr_Year_array = []
total_pop_array = []
for i in mort_d[key]:
    for j in fert_d[key]:
        
        max_lnmrdf_row = country_lnmrdf[country_lnmrdf['Year'] == max_year].drop(columns=['Country', 'Year']).values
        max_lnfrdf_row = country_lnfrdf[country_lnfrdf['Year'] == max_year].drop(columns=['Country', 'Year']).values
        max_pdf_row = country_pdf[country_pdf['Year'] == max_year].drop(columns=['Country', 'Year', 'Pop1']).values[0]
        curr_Year = max_year
        population_array = []
        population_array.append(max_pdf_row[:5].sum())
        
        for ele in max_pdf_row[5:]:
            population_array.append(ele)
        
        population_array = np.array(population_array)
        
        for year in range (0, year_range-1):
            
            next_lnmrdf = i[year]
            next_lnfrdf = j[year]
            
            max_lnmrdf_row += next_lnmrdf
            actually_mr = np.diag(np.exp(max_lnmrdf_row)[0]) 
            max_lnfrdf_row += next_lnfrdf
            actually_fr = np.exp(max_lnfrdf_row)[0][0]

            total_pop = population_array.sum()
    
            deaths = population_array * actually_mr
            deaths = np.nan_to_num(deaths.diagonal()) 
            total_deaths = deaths.sum() 
            
            births = total_pop * actually_fr
            
            population_array[0] += births
            population_array -= deaths
            
            for ele in range(len(population_array)):
                if population_array[ele] < 0:
                    population_array[ele] = 0
            
            aged = population_array / 5
            population_array -= aged
            
            for ele in range (len(aged)-1):
                population_array[ele+1] += aged[ele]
                
            population_array = np.nan_to_num(population_array)  
            population_array[population_array == inf] = 0
            
            
            total_pop = population_array.sum()
            curr_Year += 1
            
            if total_pop == inf or total_pop == 0:
                break
            else:
                curr_Year_array.append(curr_Year)
                total_pop_array.append(total_pop)
                
            # plt.scatter(curr_Year, total_pop, color='blue')
            
    
        
  


total_pop_array = np.array(total_pop_array)
curr_Year_array = np.array(curr_Year_array)

q1 = np.quantile(total_pop_array, 0.25)
q3 = np.quantile(total_pop_array, 0.75)

iqr = q3 - q1
upperBound = q3 + 1.5 * iqr
lowerBound = q1 - 1.5 * iqr

indices = np.where((total_pop_array < lowerBound) | (total_pop_array > upperBound))[0]
indices_filter = [z for z,n in enumerate(total_pop_array) if z not in indices]

fixed_total_pop_array = total_pop_array[indices_filter]
fixed_curr_Year_array = curr_Year_array[indices_filter]

plt.scatter(fixed_curr_Year_array, fixed_total_pop_array, color='blue', alpha=0.5)